### Answering Business Questions Using SQL
#### Introduction and Schema Diagram:

In this project, we are going to pratice using our SQL skills to answer businnes questions.
 
We'll use The Chinook database that is provided as a SQLite database file called chinook.db.
A copy of the database schema is below:
![chinook.db](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)


In [25]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db
## Connect our Jupyter Notebook to our database file:    

'Connected: None@chinook.db'

 ### Overview of the Data
 Query the database to get a list of all tables and views in our database:

In [26]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ('table','view')

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Selecting Albums to Purchase
Scenario: The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|Artist Name | Genre |
|:------------|:-------|
|Regal       | Hip-Hop|
|Red Tone    | Punk  |
|Meteor and the Girls| Pop|
|Slim Jim Bites| Blues|

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We'll write a query that returns each genre, with the number of tracks sold in the USA:
 - in absolute numbers and
 - in percentages.

In [27]:
%%sql
WITH track_sold_usa AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
      INNER JOIN customer c
         ON i.customer_id = c.customer_id
      WHERE c.country = 'USA')
    
SELECT  g.name genre,
        COUNT(tsu.invoice_line_id) AS track_sold,
        ROUND(CAST(COUNT(tsu.invoice_line_id) AS FLOAT)/
        (SELECT COUNT(*) FROM track_sold_usa),2) AS percentage_solt
  FROM  track_sold_usa tsu  
 INNER  JOIN track t
    ON  tsu.track_id = t.track_id 
 INNER JOIN album a 
    ON  a.album_id = t.album_id
 INNER JOIN genre g
    ON  g.genre_id = t.genre_id
 GROUP  BY 1
 ORDER  BY 2 DESC
 LIMIT  10;

Done.


genre,track_sold,percentage_solt
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Latin,22,0.02
Pop,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


based in the data that we have, like we can see, the genre which sell the best in the USA and the artits album that we should chose are: 

 |Artist Name     |Genre  |
 |:------------|:-----:|
 |Red Tone      |Punk   |
 |Slim Jim Bites|Blues  |
 |Meteor and the Girls|Pop|
 
 It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.
 

### Analyzing Employee Sales Performance

Scenario: Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.



In [28]:
%%sql
WITH support_sales_customer  AS 
(SELECT i.customer_id,
        c.support_rep_id,
        COUNT(i.total) total_sales,
        SUM(i.total) total
   FROM customer c
  INNER JOIN invoice i 
     ON c.customer_id = i.customer_id
  GROUP BY 1,2)

SELECT e.first_name ||' '||e.last_name name_employee,
       e.hire_date,
       COUNT(ssc.total) total_sales,
       SUM(ssc.total)  total 
  FROM employee e
 INNER JOIN support_sales_customer ssc
    ON ssc.support_rep_id = e.employee_id
 GROUP BY 1,2;   
  

Done.


name_employee,hire_date,total_sales,total
Jane Peacock,2017-04-01 00:00:00,21,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,20,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,18,1393.92


We can see that the best seller amongh employees is Jane Peacock, but it can be due by the different  time hire in the company...

### Analyzing Sales by Country

Our next task is to analyze the sales data for customers from each different country. we have been given guidance to use the `country` value from the customers table, and ignore the country from the billing address in the `invoice` table. 

In particular, we have been directed to calculate data, for each country, on the:

 - total number of customers
 - total value of sales
 - average value of sales per customer
 - average order value

In [32]:
%%sql
WITH countries_or_other AS 
(
 SELECT 
     CASE
        WHEN ( 
              SELECT COUNT(*)
                FROM customer
               WHERE country = c.country
             ) = 1 THEN 'Other'
        ELSE c.country
    END country,
    c.customer_id,
    il.*
  FROM invoice_line il
 INNER JOIN invoice i ON i.invoice_id = il.invoice_id
 INNER JOIN customer c ON c.customer_id = i.customer_id )  

SELECT 
     country,
     customers,
     total_sales,
     sales_avg_customer, 
    avg_order
FROM
   (
    SELECT
         country,
         COUNT(distinct customer_id) customers,
         SUM(unit_price) As total_sales,
         ROUND(SUM(unit_price) / COUNT(distinct customer_id),2) sales_avg_customer,
         ROUND(SUM(unit_price) / COUNT(distinct invoice_id),2)  avg_order,
         
        CASE
           WHEN country = 'Other' THEN 1
           ELSE 0
        END sort  
    FROM countries_or_other 
   GROUP BY country
   ORDER BY sort ASC, total_sales DESC); 

Done.


country,customers,total_sales,sales_avg_customer,avg_order
USA,13,1040.490000000008,80.04,7.94
Canada,8,535.5900000000034,66.95,7.05
Brazil,5,427.68000000000245,85.54,7.01
France,5,389.0700000000021,77.81,7.78
Germany,4,334.6200000000016,83.66,8.16
Czech Republic,2,273.24000000000103,136.62,9.11
United Kingdom,3,245.52000000000078,81.84,8.77
Portugal,2,185.13000000000022,92.57,6.38
India,2,183.1500000000002,91.58,8.72
Other,15,1094.9400000000085,73.0,7.45


We can see that base on the `average_order` the countrys that seem interesting to investing or known more about are:
 - Czech Republic
 - Unite Kingdom
 - India

### Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

 - purchase a whole album
 - purchase a collection of one or more individual tracks.
 
Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In order to risolve this, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

In [30]:
%%sql
WITH invoice_first_track AS
    (
     SELECT 
         invoice_id,
         MIN(track_id) first_track_id
     FROM invoice_line
     GROUP BY 1
    )

SELECT album_purchase,
       COUNT(invoice_id) number_of_invoices,
       ROUND((CAST(COUNT(invoice_id) AS FLOAT) / (
                                           SELECT COUNT(*) 
                                           FROM invoice
                                           )) * 100, 2) "percent (%)"                              
FROM (
      SELECT 
          ift.*,
          CASE 
              WHEN
                  (
                   SELECT t.track_id
                   FROM track t
                   WHERE t.album_id = (
                                       SELECT t2.album_id
                                       FROM track t2
                                       WHERE t2.track_id = ift.first_track_id
                                       )
                                       
                    EXCEPT
                    
                    SELECT il2.track_id
                    FROM invoice_line il2
                    WHERE il2.invoice_id = ift.invoice_id  
                   ) IS NULL
                AND 
                   (
                    SELECT il2.track_id
                    FROM invoice_line il2
                    WHERE il2.invoice_id = ift.invoice_id
                    
                    EXCEPT
                    
                    SELECT t.track_id 
                    FROM track t 
                    WHERE t.album_id = (
                                        SELECT t2.album_id
                                        FROM track t2
                                        WHERE t2.track_id = ift.first_track_id
                                        ) 
                    ) IS NULL
                THEN "yes"
                ELSE "no"
            END AS "album_purchase"
            FROM invoice_first_track ift            
    )
    
GROUP by album_purchase;

Done.


album_purchase,number_of_invoices,percent (%)
no,500,81.43
yes,114,18.57


Base on the data the numbers of album purchases is about 18.6%. 
So, my recommendation is that the Chinook store should not continue to buy full albums, instead they must considering to purchase only the most popular tracks from each album from record companies.

### Next step:
some more business questions:
 -  Which artist is used in the most playlists?
 - How many tracks have been purchased vs not purchased?
 - Is the range of tracks in the store reflective of their sales popularity?
 - Do protected vs non-protected media types have an effect on popularity?